# Examples for maps and charts using python

Charts and maps were built based on the Trase brand guidelines, using python language (Jupyter Notebook in Conda environment). All charts were built using plotly (https://plotly.com/) graphic objects. The map was built using Foilum (https://python-visualization.github.io/folium/) and spatial layers in GeoJson. The databases are hosted in GDrive as CSV and GeoJson. 

In [ ]:
#cd C:\Users\Osvaldo Pereira\Documents\Jupyter\Figure templates
#jupyter nbconvert Maps_Charts_Python.ipynb --no-input --to html
import psycopg2
import config as cfg
import requests
import plotly as plt
import pandas as pd
pd.set_option('display.max_rows', 20)
import pandas.io.sql as sqlio
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:,.2f}".format

#--------------Custom Variables--------------------
Year = 2018
deforestation_threshold = 70.0 
#EU zone countries list (2022)
EU_countries_list = ["BE","BG","CZ","DK","DE","EE","IE","GR","ES","FR","HR","IT","CY","LV","LT","LU","HU","MT","NL","AT","PL","PT","RO","SI","SK","FI","SE"]
#Soy risk deforestation database (TRASE)
URL = 'https://drive.google.com/file/d/1WJQ9ws1Sq4g-iB7GBgQfXs4NJ3QZv6Lo/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
base_data = pd.read_csv(path)
#---------------------------------------------------

##Preparing base data for generation risk threshold indicator-----------

#1. Deannualize risk
base_data['Commodity Deforestation Risk'] = base_data['Commodity Deforestation Risk'].multiply(5)

#2. Filter with up-to-date list date of EU countries
EU_Imports_base = base_data[base_data['Country Of Import Trase ID'].isin(EU_countries_list) == True]
NonEU_imports = base_data[base_data['Country Of Import Trase ID'].isin(EU_countries_list) == False]\
.rename(columns={'rela_defo': 'NEU_DEFO'})

#3. Grouping countries by juristiction of interest
EU_Imports = EU_Imports_base.groupby(['Region Production 1 Trase ID',
                                      'Region Production 1', 
                                      'Region Production 2',
                                      'Country of production',
                                      'Biome'])['Volume', 'Commodity Deforestation Risk'].sum().reset_index()

#4. Deforestation footprint calculation (ha of deforestation/ktonnes of imported volume)
EU_Imports['defo_footprint'] = EU_Imports['Commodity Deforestation Risk'].divide(EU_Imports['Volume']\
                                                                                 .divide(1000)).round(1)
#---------------------------------------------------------------------------

#Risk classifiction of origins for EU imports based on relative measure

#1. Etsimate the relative risk of deforestation
EU_Imports["relative_deforestation"] = (EU_Imports['Commodity Deforestation Risk']\
                                        .divide(EU_Imports['Commodity Deforestation Risk'].sum()).multiply(100))
EU_Imports['cumulative_share'] = EU_Imports['relative_deforestation']\
                                        .sort_values(ascending=False).cumsum().round(1)

#2. create the risk threshold
conditions = [(EU_Imports['cumulative_share'] == 100.0), # No-risk
              (EU_Imports['cumulative_share'] < 100.0) & (EU_Imports['cumulative_share'] >= 99.0),#low-risk
              (EU_Imports['cumulative_share'] < 99.0) & (EU_Imports['cumulative_share'] > deforestation_threshold),#Medium-risk
              (EU_Imports['cumulative_share'] <= deforestation_threshold)]#High-risk

#3. Risk classes as int 
risk_classes = ['1', '2', '3', '4']

#4. Assign risk classes
EU_Imports['Risk_score'] = np.select(conditions, risk_classes)

In [ ]:
#Prepare and concat spatial data------------------
import geopandas as gpd
from geopandas import GeoDataFrame
import json
import urllib.request, json 
from pandas.io.json import json_normalize
import folium
import branca.colormap as cm
from folium.plugins import Fullscreen

#Open Json file from GDrive (Juristictions for folium map)
URL1 = 'https://drive.google.com/file/d/1WKYZ06zftHskrvJGSsqX7e-BtAOJy3_O/view?usp=sharing'
path1 = 'https://drive.google.com/uc?export=download&id='+URL1.split('/')[-2]
with urllib.request.urlopen(path1) as url:
    geo_json_data = gpd.read_file(url.read().decode())

#Spatial table: concat dataframe with spatial data
spatial_table = pd.concat([geo_json_data.set_index('Geocod'), EU_Imports.set_index('Region Production 1 Trase ID')], 
                          axis = 1).reset_index(drop=True).dropna()
spatial_table = spatial_table[spatial_table['Region Production 1'].notna()]

spatial_table = spatial_table[['geometry', 'Region Production 1', 'Region Production 2', 'Country of production',
                               'Biome','Volume', 'Commodity Deforestation Risk', 'defo_footprint', 
                               'relative_deforestation', 'cumulative_share', 'Risk_score']] 

spatial_table = spatial_table.rename({'Region Production 1': 'Municipality/District',
                                      'Region Production 2': 'State/Province', 
                                      'Volume': 'Volume (tonnes)', 
                                      'Commodity Deforestation Risk': 'Soy deforestation risk (ha)', 
                                      'defo_footprint': 'Deforestation footprint (ha/ktonnes of soy)', 
                                      'relative_deforestation': 'Relative deforestation (%)', 
                                      'Risk_score': 'Risk_score'}, axis=1)
spatial_table['Risk_score'] = spatial_table['Risk_score'].astype(int)
spatial_table['Volume (tonnes)'] = spatial_table['Volume (tonnes)'].round(2)
spatial_table['Soy deforestation risk (ha)'] = spatial_table['Soy deforestation risk (ha)'].round(2)
spatial_table['Relative deforestation (%)'] = spatial_table['Relative deforestation (%)'].round(2)
spatial_table['Risk_score'] = spatial_table['Risk_score'].astype(int)

#Risk classes as strings
conditions = [(spatial_table['Risk_score'] == 1),
              (spatial_table['Risk_score'] == 2),
              (spatial_table['Risk_score'] == 3),
              (spatial_table['Risk_score'] == 4)
              ]

values = ['Low-risk', 'Residual', 'Medium risk', 'High risk']

#Final spatial table
spatial_table['Risk categories'] = np.select(conditions, values)

## 1. Pie chart, scatter chart, and Funnel chart

These are the most commonly used charts for communication on slides. I didn’t include all charts, but the layout options can be easily transferred to other charts available in plotly.  

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly.io as pio
pio.renderers.default='notebook'

#DATA PREPARATION-----------------------------------------------------------
biome_embedded_deforestation = spatial_table.groupby(['Biome'], as_index=False).agg({'Volume (tonnes)':'sum', 
                                                                                     'Relative deforestation (%)': 'sum',
                                                                                     'Soy deforestation risk (ha)': 'sum', 
                                                                                     'Deforestation footprint (ha/ktonnes of soy)' : 'mean'})

state_embedded_deforestation = spatial_table.groupby(['Country of production','State/Province', 'Biome'], as_index=False).agg({'Volume (tonnes)':'sum', 
                                                                                     'Relative deforestation (%)': 'sum',
                                                                                     'Soy deforestation risk (ha)': 'sum', 
                                                                                     'Deforestation footprint (ha/ktonnes of soy)' : 'mean'})

risk_embedded_deforestation = spatial_table.groupby(['Risk categories'], as_index=False).agg({'Volume (tonnes)':'sum', 
                                                                                     'Relative deforestation (%)': 'sum',
                                                                                     'Soy deforestation risk (ha)': 'sum', 
                                                                                     'Deforestation footprint (ha/ktonnes of soy)' : 'mean'})

biome_embedded_deforestation['Deforestation footprint (ha/ktonnes of soy)']=biome_embedded_deforestation['Deforestation footprint (ha/ktonnes of soy)'].round(1)

EU_Imports_base["relative_deforestation"] = (EU_Imports_base['Commodity Deforestation Risk'].divide(EU_Imports_base['Commodity Deforestation Risk'].sum()).multiply(100))
EU_Imports_base['cumulative_share'] = EU_Imports_base['relative_deforestation'].sort_values(ascending=False).cumsum()

by_counrty = EU_Imports_base.groupby(['Country Of Import'], as_index=False).agg({'Volume':'sum', 
                                                                                 'relative_deforestation':'sum',
                                                                                 'Commodity Deforestation Risk':'sum'}).sort_values(by=['Commodity Deforestation Risk'], ascending=False).head(12)

by_counrty['Deforestation footprint'] = by_counrty['Commodity Deforestation Risk'].divide(by_counrty['Volume'].divide(1000)).round(2)
by_counrty['Deforestation footprint'] = by_counrty['Deforestation footprint'].sort_values(ascending=False)
by_counrty = by_counrty.head(13)
#---------------------------------------------------------------------------------

#Configure Charts-----------------------------------------------------------------

#Subplots----
fig = make_subplots(
    column_widths=[0.3, 0.3],
    horizontal_spacing=0.23,
    vertical_spacing=0.15,
    rows=2, cols=2,
    specs=[[{"type": "domain"}, {"type": "xy"}], [{"type": "treemap"}, {"type": "bar"}]],
    subplot_titles=("<b>a. Pie Chart</b>",
                    '<b>b. Scatter chart with labels</b>',
                    "<b>c. Sunburst chart</b>", 
                    '<b>d. Funnel chart</b>'))
#-------------

#PIE CHART---
fig.add_trace(go.Pie(labels = risk_embedded_deforestation['Risk categories'], 
           values = risk_embedded_deforestation['Volume (tonnes)'],
           textposition='outside',
           insidetextorientation='horizontal',
           hole = 0.65,#
           showlegend = False,
           textinfo='label+percent',
           textfont=dict(
              family="DM Sans",
              size=16,
              color='#839A8C'),
           texttemplate='<b>%{label}</b><br>%{percent}',
           hovertemplate='<b>%{label}</b> <br>Embedded deforestation: %{value} ha',
           name = 'Pie chart',
           marker = dict(colors= ['#FF66CC', '#E9EF00', '#E27227', '#4E6356'], line=dict(color='#ffffff', width=0)
           )),
           row=1, col=1)
#------------

#SCATTER CHART--
fig.add_trace(go.Scatter(
              x=biome_embedded_deforestation['Volume (tonnes)'],
              y=biome_embedded_deforestation['Soy deforestation risk (ha)'],
              showlegend=False,
              mode='markers+text',
              marker=dict(size=biome_embedded_deforestation['Deforestation footprint (ha/ktonnes of soy)'].multiply(8), 
                          color=['#00D099', '#2EE8C5', '#FF8600', '#8B87FF', '#2548D8'],
                          opacity=0.6,
                          line=dict(color='#a0a0a0', width=2)),
              name = 'Figure c',
              text = biome_embedded_deforestation['Biome'].astype(str) + ': ' + biome_embedded_deforestation['Deforestation footprint (ha/ktonnes of soy)'].astype(str),
              textposition=['top center', 'middle right', 'top center', 'middle center', 'bottom center'], 
              textfont=dict(family="DM Sans", size=13, color="#000000")
              ), row=1, col=2)
#---------------

#Sunburst Chart--
sunburst_table = spatial_table.groupby(['Country of production', 'State/Province', 'Municipality/District'], as_index=False)\
.agg({'Soy deforestation risk (ha)': 'sum', 'Volume (tonnes)': 'sum'})

sunburst_table = sunburst_table[sunburst_table['Soy deforestation risk (ha)'] >= 1]                                                                                                                                  
sunburst_table = sunburst_table[['Country of production', 'State/Province', 'Municipality/District', 'Soy deforestation risk (ha)', 'Volume (tonnes)']]
sunburst_table['cor'] = (sunburst_table['Soy deforestation risk (ha)']-sunburst_table['Soy deforestation risk (ha)'].min())/(sunburst_table['Soy deforestation risk (ha)'].max()-sunburst_table['Soy deforestation risk (ha)'].min())*10

df = sunburst_table

levels = ['Municipality/District', 'State/Province', 'Country of production'] # levels used for the hierarchical chart
color_columns = 'cor'
value_column = 'Soy deforestation risk (ha)'

def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = ''
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns] 
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees

df_all_trees = build_hierarchical_dataframe(df, levels, value_column, color_columns)

average_score = sunburst_table['cor'].sum()

fig.add_trace(go.Sunburst(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    textinfo='label+percent root',
    textfont=dict(
        family="DM Sans",
        color='#658270'),
    texttemplate="<b>%{label}</b><br>%{percentRoot}",
    maxdepth=2,
    insidetextorientation='radial',
    marker=dict(
        colors=df_all_trees['color'],
        line=dict(color='#ffffff', width=2),
        colorscale=['#BBFFEC', '#FF6A5F'],
        cmid=average_score, 
        cmax=90,
        cmin=0),
    hovertemplate='<b>%{label}</b> <br>Embedded deforestation: %{value} ha',
    name=''
    ), 
    row=2, col=1)
#------------

#FUNNEL CHART--
fig.add_trace(go.Funnel(name = 'Funnel chart',
                        y = by_counrty['Country Of Import'],
                        x = by_counrty['Deforestation footprint'],
                        textinfo = "value", 
                        hovertemplate='<b>%{y}</b> <br>Footprint: %{x}',
                        texttemplate="<b>%{value}</b>",
                        textfont=dict(
                         family="Space Mono",
                         size=20,
                         color='#798B85'),
                        opacity = 1,
                        showlegend = False,
                        marker = {"color":  '#F8F4EA', "line": {"width": 0.5, "color": '#798B85'}}
                        ),row=2, col=2)
#--------------

#FIGURE GENERAL SETTING AND AXIS CONFIGURE-----------------------
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor = "white",
                  yaxis=dict(ticks='outside', 
                              tick0=0, 
                              tickwidth=1,
                              tickcolor='#839A8C', 
                              linecolor='#839A8C', 
                              tickfont = {"color":'#839A8C', "size":12, "family":"DM Sans"},
                              linewidth=2.0, 
                              title='<b>Volume (tonnes)</b>'),
                  xaxis1=dict(ticks='outside', 
                              tick0=0, 
                              tickwidth=1,
                              tickcolor='#839A8C', 
                              linecolor='#839A8C', 
                              linewidth=2.0, 
                              title='<b>Volume (tonnes)</b>'),
                  yaxis1=dict(ticks='outside', 
                              tick0=0, 
                              tickwidth=1,
                              tickfont = {"color":'#798B85', "size":14, "family":"Space Mono"},
                              tickcolor='#839A8C', 
                              linecolor='#839A8C', 
                              linewidth=2.0, 
                              title='<b>Soy deforestation (ha)</b>'),
                  yaxis2=dict(linecolor='#839A8C', 
                              linewidth=2.0, 
                              categoryorder='total descending',
                              tickfont = {"color":'#839A8C', "size":15, "family":"DM Sans"},
                              tickprefix="<b>",
                              ticksuffix ="</b><br>"),
                  hoverlabel=dict(bgcolor='#ffffff', font_size=12, font_family="DM Sans"))
                  
fig.update_xaxes(
        title_font = {"size": 16, "color": '#839A8C', "family":"DM Sans"},
        title_standoff = 10, 
        tickfont = {"color":'#798B85', "size":14, "family":"Space Mono"})

fig.update_yaxes(
        title_font = {"size": 16, "color": '#839A8C', "family":"DM Sans"},
        title_standoff = 0)

fig.layout.annotations[0].update(y=1.05, x=0.02)
fig.layout.annotations[1].update(y=1.05, x=0.72)
fig.layout.annotations[2].update(y=0.44, x=0.02)
fig.layout.annotations[3].update(y=0.44, x=0.68)

for i in fig['layout']['annotations']:
    i['font'] = dict(family="DM Sans", size=18,color='#31464E')
 
fig['layout'].update(height=1000)
#------------------------------------------------

fig.show()

## 2. Stacked bar charts

In [ ]:
#Prepare charts for countries--------------------------------------------------
by_biome_counrty = EU_Imports_base.groupby(['Biome', 'Country Of Import'], as_index=False).agg({'Volume':'sum', 
                                                                                                'relative_deforestation':'sum',
                                                                                                'Commodity Deforestation Risk':'sum'})

by_biome_counrty = by_biome_counrty[['Country Of Import', 'Biome','Commodity Deforestation Risk']]

by_biome_counrty = pd.pivot_table(by_biome_counrty, 
                            values='Commodity Deforestation Risk', 
                            index=['Country Of Import'], 
                            columns=['Biome'], 
                            aggfunc=np.sum).reset_index().rename_axis(None, axis=1).fillna(0)

by_biome_counrty = by_biome_counrty.sort_values(by=['Amazon'], ascending=False).head(12)

by_biome_counrty_prop = by_biome_counrty
by_biome_counrty_prop['Total'] = by_biome_counrty_prop.sum(axis=1)
by_biome_counrty_prop['Atlantic Forest %'] = by_biome_counrty_prop['Altantic Forest'] / by_biome_counrty_prop['Total'] * 100
by_biome_counrty_prop['Amazon %'] = by_biome_counrty_prop['Amazon'] / by_biome_counrty_prop['Total'] * 100
by_biome_counrty_prop['Cerrado %'] = by_biome_counrty_prop['Cerrado'] / by_biome_counrty_prop['Total'] * 100
by_biome_counrty_prop['Dry Chaco %'] = by_biome_counrty_prop['Dry Chaco'] / by_biome_counrty_prop['Total'] * 100
by_biome_counrty_prop['Humid Chaco %'] = by_biome_counrty_prop['Humid Chaco'] / by_biome_counrty_prop['Total'] * 100
by_biome_counrty_prop = by_biome_counrty_prop[['Country Of Import', 'Atlantic Forest %', 'Amazon %', 'Cerrado %', 'Dry Chaco %', 'Humid Chaco %']]
#---------------------------------------------------------------------------------

#Configure Charts-----------------------------------------------------------------
#Subplots----
fig1 = make_subplots(
    horizontal_spacing=0.23,
    rows=1, cols=2,
    specs=[[{"type": "xy"}, {"type": "bar"}]],
    subplot_titles=("<b>a. Stacked bar chart</b>",
                    '<b>b. 100% stacked bar chart</b>'))
#-------------

#-------------

#STACKED BAR CHART---
by_biome_by_country0 = go.Figure(data=[go.Bar(name='Amazon', 
                                           x=by_biome_counrty['Amazon'], 
                                           y=by_biome_counrty['Country Of Import'],
                                           hovertemplate='<b>%{y}</b> <br>Defo. risk: %{x:.2f} ha',
                                           orientation='h',
                                           marker_color='#61F261', 
                                           showlegend=False),
                                 go.Bar(name='Atlantic Forest', 
                                           x=by_biome_counrty['Altantic Forest'], 
                                           y=by_biome_counrty['Country Of Import'],
                                           hovertemplate='<b>%{y}</b> <br>Defo. risk: %{x:.2f} ha',
                                           orientation='h',
                                           marker_color='#00D099', 
                                           showlegend=False), 
                                 go.Bar(name='Cerrado', 
                                           x=by_biome_counrty['Cerrado'], 
                                           y=by_biome_counrty['Country Of Import'],
                                           hovertemplate='<b>%{y}</b> <br>Defo. risk: %{x:.2f} ha',
                                           orientation='h',
                                           marker_color='#FF8600', 
                                           showlegend=False),
                                 go.Bar(name='Humid Chaco', 
                                           x=by_biome_counrty['Humid Chaco'], 
                                           y=by_biome_counrty['Country Of Import'],
                                           hovertemplate='<b>%{y}</b> <br>Defo. risk: %{x:.2f} ha',
                                           orientation='h',
                                           marker_color='#8B87FF', 
                                           showlegend=False), 
                                 go.Bar(name='Dry Chaco', 
                                           x=by_biome_counrty['Dry Chaco'], 
                                           y=by_biome_counrty['Country Of Import'], 
                                           hovertemplate='<b>%{y}</b> <br>Defo. risk: %{x:.2f} ha',
                                           orientation='h',
                                           marker_color='#D17EFC', 
                                           showlegend=False)])

by_biome_by_country = go.Figure(data=by_biome_by_country0)

for t in by_biome_by_country.data:
    fig1.append_trace(t, row=1, col=1)
#------------

#STACKED PROPORTIONAL BAR CHART---
by_biome_counrty_prop0 = go.Figure(data=[go.Bar(name='Amazon', 
                                           x=by_biome_counrty_prop['Amazon %'], 
                                           y=by_biome_counrty_prop['Country Of Import'],
                                           hovertemplate='<b>%{y}</b> <br>Prop: %{x:.1f}%',
                                           orientation='h',
                                           marker_color='#61F261', 
                                           showlegend=True),
                                 go.Bar(name='Atlantic Forest', 
                                           x=by_biome_counrty_prop['Atlantic Forest %'], 
                                           y=by_biome_counrty_prop['Country Of Import'],
                                           hovertemplate='<b>%{y}</b> <br>Prop: %{x:.1f}%',
                                           orientation='h',
                                           marker_color='#00D099', 
                                           showlegend=True), 
                                 go.Bar(name='Cerrado', 
                                           x=by_biome_counrty_prop['Cerrado %'], 
                                           y=by_biome_counrty_prop['Country Of Import'],
                                           hovertemplate='<b>%{y}</b> <br>Prop: %{x:.1f}%',
                                           orientation='h',
                                           marker_color='#FF8600', 
                                           showlegend=True),
                                 go.Bar(name='Humid Chaco', 
                                           x=by_biome_counrty_prop['Humid Chaco %'], 
                                           y=by_biome_counrty_prop['Country Of Import'],
                                           hovertemplate='<b>%{y}</b> <br>Prop: %{x:.1f}%',
                                           orientation='h',
                                           marker_color='#8B87FF', 
                                           showlegend=True), 
                                 go.Bar(name='Dry Chaco', 
                                           x=by_biome_counrty_prop['Dry Chaco %'], 
                                           y=by_biome_counrty_prop['Country Of Import'], 
                                           hovertemplate='<b>%{y}</b> <br>Prop: %{x:.1f}%',
                                           orientation='h',
                                           marker_color='#D17EFC', 
                                           showlegend=True)])

by_biome_counrty_prop = go.Figure(data=by_biome_counrty_prop0)

for t in by_biome_counrty_prop.data:
    fig1.append_trace(t, row=1, col=2)
    
#------------
fig1.update_layout(paper_bgcolor='rgba(0,0,0,0)',
                   plot_bgcolor = "white",
                   barmode='stack',
                   bargap=0.05,
                   xaxis=dict(ticks='outside', 
                              tick0=0, tickwidth=1,
                              tickcolor='#839A8C', 
                              linecolor='#839A8C', 
                              linewidth=2.0, 
                              showgrid=True, 
                              gridwidth=0.5, 
                              gridcolor='#D2D9D7',
                              title='<b>Deforestation (ha)</b>'),
                   yaxis=dict(categoryorder='total ascending',
                              linecolor='#839A8C', 
                              linewidth=2.0,
                              tickprefix="<b>",
                              ticksuffix ="</b><br>"),
                   xaxis2=dict(ticks='outside', 
                              tick0=0, 
                              tickwidth=1,
                              dtick=25,
                              tickcolor='#839A8C', 
                              linecolor='#839A8C', 
                              linewidth=2.0, 
                              title='<b>Deforestation (%)</b>', 
                              ticksuffix ="%"),
                   yaxis2=dict(categoryorder='total ascending',
                              linecolor='#839A8C', 
                              linewidth=2.0,
                              tickprefix="<b>",
                              ticksuffix ="</b><br>"),
                   hoverlabel=dict(bgcolor='#ffffff', 
                                   font_size=12, 
                                   font_family="DM Sans"),
                   legend=dict(orientation="h",
                               yanchor="bottom",
                               y=-0.3,
                               xanchor="right",
                               x=0.83,
                               itemclick=False,
                               font=dict(
                                family="DM Sans",
                                size=12,
                                color="#4D655E")))

fig1.update_xaxes(
        title_font = {"size": 15, "color": '#839A8C', "family":"DM Sans"},
        title_standoff = 10, 
        tickfont = {"color":'#798B85', "size":13, "family":"Space Mono"})

fig1.update_yaxes(
        title_font = {"size": 15, "color": '#839A8C', "family":"DM Sans"},
        title_standoff = 0, 
        tickfont = {"color":'#839A8C', "size":14, "family":"DM Sans"})

fig1.layout.annotations[0].update(y=1.07, x=0.0)
fig1.layout.annotations[1].update(y=1.07, x=0.63)
fig1.for_each_trace(lambda t: t.update(name = '<b>' + t.name +'</b>'))
fig1.layout.legend.itemsizing = 'trace'


fig1.show()

## 3. Folium map

Map created using real datasets from Trase in Argentina, Brazil, and Paraguay. The goal was to represent different layers (biome, states, countries, data layer), so we can evaluate how these different layers are being presented in the map. 

In [ ]:
def getcolor(feature):
    if feature['properties'] ['Risk_score'] == 1:
        return '#4E6356'
    if feature['properties'] ['Risk_score'] == 2:
        return '#E9EF00'
    if feature['properties'] ['Risk_score'] == 3:
        return '#E27227'
    if feature['properties'] ['Risk_score'] == 4:
        return '#FF66CC'
    else:
        return '#ffffff'
    
m = folium.Map(location=[-18.007071, -51.603937], 
               zoom_start=4, tiles=None, 
               control_scale = True,
               zoom_control=False,
               scrollWheelZoom=True,
               dragging=True)

#

URL_state = 'https://drive.google.com/file/d/1WMH_lkx1Bws5grq_1Eqwqh8-khBhPS5t/view?usp=sharing'
path_state = 'https://drive.google.com/uc?export=download&id='+URL_state.split('/')[-2]
with urllib.request.urlopen(path_state) as url:
    state_layer = gpd.read_file(url.read().decode())
    
URL_country = 'https://drive.google.com/file/d/1WUqVSyMjWvwRlTEIoojO7OJDnDP1IE9w/view?usp=sharing'
path_country = 'https://drive.google.com/uc?export=download&id='+URL_country.split('/')[-2]
with urllib.request.urlopen(path_country) as url:
    countries_layer = gpd.read_file(url.read().decode())
    
URL_white = 'https://drive.google.com/file/d/1WhdYIfMGn8jmSJmFF6AFyHT0BVZObhuq/view?usp=sharing'
path_white = 'https://drive.google.com/uc?export=download&id='+URL_white.split('/')[-2]
with urllib.request.urlopen(path_white) as url:
    white_basemap = gpd.read_file(url.read().decode())
    
URL_biome = 'https://drive.google.com/file/d/1XOr2eXgHxfp7pqP7u9K3gI7iY_UNzy7E/view?usp=sharing'
path_biome = 'https://drive.google.com/uc?export=download&id='+URL_biome.split('/')[-2]
with urllib.request.urlopen(path_biome) as url:
    biome_layer = gpd.read_file(url.read().decode())

state_style = {'fillColor': '#00000000', 'color': '#ffffff', 'weight': 1.5} 
biome_style = {'fillColor': '#00000000', 'color': '#404040', 'weight': 1.3} 
countries_style = {'fillColor': '#E2EAE7', 'fillOpacity': 1, 'color': '#00000000', 'weight': 0} 
countries_style_line = {'fillColor': '#00000000', 'color': '#a0a0a0', 'weight': 1} 
white_basemap_style = {'fillColor': '#ffffff', 'fillOpacity': 1,'color': '#ffffff', 'weight': 0} 

folium.GeoJson(white_basemap,name='Blank basemap',style_function=lambda x:white_basemap_style, control=False).add_to(m)
folium.GeoJson(countries_layer,name='Countries',style_function=lambda x:countries_style, control=False).add_to(m)
folium.GeoJson(spatial_table, name='Data layer',smooth_factor = 0, style_function = lambda feature: {
        'fillColor': getcolor(feature),
        'weight': 0,
        'fillOpacity': 1,
}).add_to(m)
folium.GeoJson(state_layer,name='States',style_function=lambda x:state_style, control=False).add_to(m)
folium.GeoJson(countries_layer,name='Country Boundaries',style_function=lambda x:countries_style_line, control=False).add_to(m)
folium.GeoJson(biome_layer,name='Biome boundaries',style_function=lambda x:biome_style, control=True).add_to(m)

Fullscreen().add_to(m)
folium.LayerControl().add_to(m)

#----------------------Add hover functionality--------------------------------
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#00000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

NIL = folium.features.GeoJson(
    data = spatial_table,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Municipality/District', 'State/Province', 'Volume (tonnes)', 'Soy deforestation risk (ha)', 'Deforestation footprint (ha/ktonnes of soy)', 'Relative deforestation (%)'],
        aliases=['Municipality/District', 'State/Province', 'Volume (tonnes)', 'Soy deforestation risk (ha)', 'Deforestation footprint (ha/ktonnes of soy)', 'Relative deforestation (%)'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

#Legend: Create an HTML object as a legend (Folium option is not configurable)------------
from branca.element import Template, MacroElement
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px solid grey; background-color:#00000000;
     border-radius:6px; padding: 16px; font-size:16px; font-family:DM Sans;right: 30px; bottom: 30px;'>

<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FF66CC;opacity:1;border-radius:50%;height:17px;width:17px;margin-bottom:3px;border-width: 0px;'></span><b>1. High-risk</b><br></li>
    <li><span style='background:#E27227;opacity:1;border-radius:50%;height:17px;width:17px;margin-bottom:3px;border-width: 0px;'></span><b>2. Medium-risk</b><br></li>
    <li><span style='background:#E9EF00;opacity:1;border-radius:50%;height:17px;width:17px;margin-bottom:3px;border-width: 0px;'></span><b>3. Low-risk</b><br></li>
    <li><span style='background:#4E6356;opacity:1;border-radius:50%;height:17px;width:17px;margin-bottom:3px;border-width: 0px;'></span><b>4. No-risk</b><br></li>
    <li><span style='background:#00000000;opacity:1;border-radius:50%;height:17px;width:17px;margin-bottom:3px;border-color:#404040; border-width: 3px;'></span><b>Biome boundaries</b><br></li>
    
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
m.get_root().add_child(macro)
#-----------------------------------------------------------------------------------------------------------

m